In [1]:
################ convert pdf to images ########################

from pdf2image import convert_from_path
import os

def pdf_to_images(pdf, filename):
    pdf = pdf.replace('\\', '\\\\')
    # print(pdf)
    images = convert_from_path(pdf)
    for i in range(len(images)):
        images[i].save('C:/matsya/images_apr_24/{}_page{}'.format(filename, i) +'.jpg', 'JPEG')

path = "C:\\matsya\\Apr-24"

#store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))


# print all the file names
for file in filelist:
    filename = os.path.split(file)[-1].split('.pdf')[0]
    if ' ' in filename:
        filename = filename.replace(' ', '_')
    # print(filename)
    try:
        pdf_to_images(file, filename)  
    except:
         pass

In [2]:
######## remove unwanted files #############

import glob

count = 0
final_images = []
for image in glob.glob('C:/matsya/images_apr_24/*'):
    count += 1
    if 'ms_' in image.lower() or 'mr_' in image.lower():
        continue
    final_images.append(image)

print(count)
print(len(final_images))

108
73


In [3]:
#################### parse output of original images ######################

import glob

for image in final_images:
    filename = os.path.split(image)[-1].split('.jpg')[0]
    psm_values = [4]
    for psm in psm_values:
        parsed_output = 'C:/matsya/output_apr_24/'+filename+'_'+str(psm)
        command = 'tesseract {} {} -l eng --psm {}'.format(image, parsed_output, psm)
        os.system(command)

In [4]:
################### ensemble the output files #####################

import glob
import os

def ensemble_output():

    for file in glob.glob('C:/matsya/output_apr_24/*.txt'):
        filename = os.path.split(file)[-1].split('.txt')[0]
        ind = filename.index('page')
        filename = filename[:ind+5]
        # print(filename)
        with open(file, encoding="utf-8") as fp:
                # read the file
                text = fp.read()
                # open new file and append text of files for the same name 
                with open('C:/matsya/ensemble_files_apr_24/{}.txt'.format(filename), 'a') as file:
                    # to separate the text of multiple configuration using ---
                    file.write(text+"\n---"+"\n")

ensemble_output()

In [5]:
## extract key value pairs

import glob
import os
import datetime
import re
import pandas as pd

def clean_trav_name(traveler_names):
    new_trav = []
    debar = [':', '\'', ' i', '{', '(', ' t', '\\\\', '+', '|', '4', '\\', '!']
    for tn in traveler_names:
        for ch in debar:
            if ch in tn:
                tn = tn.replace(ch, '')
        
        if 'seat' not in tn.lower():
            new_trav.append(tn.strip())
    return new_trav

# def get_traveler_names(text):
#     traveler_names = []
#     flag = 0
#     for i in range(len(text)):
#         if 'pax name' in text[i].lower() or 'tax & chrg' in text[i].lower() or 'airline' in text[i].lower():
#             for j in range(i+1, len(text)-1):
#                 if 'mr' in text[j].lower() or 'ms' in text[j].lower() or 'mrs' in text[j].lower() or 'is' in text[j].lower():
#                     words = text[j].split()
#                     for k in range(len(words)):
#                         words[k] = words[k].replace('|', '')
#                         if ('mr' in words[k].lower() or 'ms' in words[k].lower() or 'mrs' in words[k].lower() or 'is' in words[k].lower()) and len(words[k])<=4:
#                             traveler_name = ''
#                             for l in range(k, len(words)):
#                                 # if (not words[l].isalpha() or words[l].lower() == 'date') and traveler_name and 'ams' not in traveler_name.lower():
#                                 if not words[l].isalpha() and traveler_name and 'ams' not in traveler_name.lower():
#                                     traveler_names.append(traveler_name.strip('‘ '))
#                                     flag = 1
#                                     break
#                                 traveler_name += words[l]+' '
#                             # print('traveler name---> ', traveler_name)
#                 if 'sub-total' in text[j].lower() or 'processing charges' in text[j].lower() or 'terms & conditions' in text[j].lower():
#                     break
#         if flag:
#             break

#     if not traveler_names:
#         traveler_names.append('None')
    
#     new_trav_names = clean_trav_name(traveler_names)
#     return new_trav_names

def get_traveler_names(text):
    traveler_names = []
    traveler_name = ''
    flag = 0
    for i in range(len(text)):
        
        if 'pax name' in text[i].lower() or 'tax & chrg' in text[i].lower() or 'basic fare' in text[i].lower() or ('sector' in text[i].lower() and len(text[i].split())==1):
            for j in range(i+1, len(text)-1):
                if 'sac' in text[j].lower():
                    continue
                row = text[j].split()
                flag = 0
                if 'sub-total' in text[j].lower() or 'processing charges' in text[j].lower():
                    break
                for k in range(2, len(row)):
                    if row[k].isalpha():
                        for l in range(k, len(row)):
                            # print('row[l]--> ', row[l])
                            if '/' in row[l]:
                                # print('traveler name---> ', traveler_name)
                                traveler_names.append(traveler_name.strip())
                                traveler_name = ''
                                flag = 1
                                break
                            traveler_name += row[l] + ' '
                    if flag:
                        break
                   
    # print('traveler names---> ', traveler_names)
    new_trav_names = clean_trav_name(traveler_names)
    # print('new trav names---> ', new_trav_names)
    return new_trav_names


def get_start_dates(text):
    start_dates = []
    start_date = ''
    joinedtext = ' '.join(text)
    n = len(get_traveler_names(text))
    flag = 0
    months = ['jan', 'jgan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'gan', 'aapr', 'gun', 'jgun', 'gul', 'dun', 'jdul', 'dul', '0ct', 'ooct']

    if 'flight details' not in joinedtext.lower() and 'inv.dt.' not in joinedtext.lower():
        
        for line in text:
            if 'crn.dt.' in line.lower():
                start_date = line.split()[-1]
                # start_dates.append(start_date)
                start_dates.extend([start_date]*n)
                # return start_dates

    elif 'inv.dt.' in joinedtext.lower():
        
        for i in range(len(text)):
            if 'inv.dt.' in text[i].lower():
                year = text[i].split()[-1].split('-')[-1]
            if 'pax name' in text[i].lower() or 'tax & chrg' in text[i].lower() or 'airline' in text[i].lower():
                for j in range(i+1, len(text)):
                    words = text[j].split()
                    if ('seat charges' in text[j].lower() or 'meal charges' in text[j].lower() or 'extra baggage' in text[j].lower() or 'excess baggage charges' in text[j].lower()) and 'mr.' not in text[j].lower():
                        continue
                    for word in words:
                        for month in months:
                            if month in word.lower() and '-' in word:
                                flag = 1
                                start_date = word
                                start_date = start_date+'-'+year
                                start_dates.append(start_date)
                                # print('start dates---> ', start_dates)
                                break
                    if 'sub-total' in text[j].lower() or 'processing charges' in text[j].lower() or 'inr' in text[j].lower():
                        break
            if flag:
                break
            # if not flag:
            #     start_dates.append('None')
    
    # print('old start dates---> ', start_dates)
    correct_months = {'31l': '31', 'O1l':'01', 'Jgan': 'Jan', 'Jgun': 'Jun', 'gan': 'Jan', 'Aapr': 'Apr', 'gun': 'Jun', 'aApr': 'Apr', 'gul': 'Jul', 'dun': 'Jun', 'Jdul': 'Jul', 'dul': 'Jul', '0ct': 'Oct', 'Ooct': 'Oct'}
    for i in range(len(start_dates)):
        for k, v in correct_months.items():
            if k in start_dates[i]:
                start_dates[i] = start_dates[i].replace(k, v)
    # print('start dates---> ', start_dates)

    newdates = clean_date(start_dates)

    finaldates = []
    for newdate in newdates:
        finaldate = ''
        formats = ['%d-%b-%Y', '%d-%m-%Y']
        for format in formats:
            try:
                # print('NEWDATE---> ', newdate)
                finaldate = datetime.datetime.strptime(newdate, format).strftime('%Y-%m-%d')
                finaldates.append(finaldate)
            except:
                pass
    
    if not finaldates:
        finaldates.append('None')
        return finaldates
    
    # print('final dates---> ', finaldates)
    return finaldates

def clean_date(dates):
    delete_ch = [':', '_', '_:', '_\'']
    replace_ch = {'O':'0', 'Q':'0', 'l':'1', 'g':'8', 'i':'1', 'S':'5', 'o':'0'}
    newdates = []
    for date in dates:
        newdate = date
        flag = 0
        for c in date:
            if c == '-':
                flag = 1
            for d in delete_ch:
                if d in c:
                    newdate = newdate.replace(d, '')
            if not flag:
                for k, v in replace_ch.items():
                    if k in c:
                        newdate = newdate.replace(k, v)
        newdates.append(newdate)
    
    return newdates

def get_sectors(text):
    sectors = []
    travelers = get_traveler_names(text)
    l = len(travelers)
    flag = 0
    for i in range(len(text)):
        if 'sectors' in text[i].lower() or 'tax & chrg' in text[i].lower() or 'pax name' in text[i].lower() or 'flight details' in text[i].lower() or ('sector' in text[i].lower() and len(text[i].split()) == 1) or 'basic fare' in text[i].lower():
            for j in range(i+1, len(text)):
                words = text[j].split()
                flag = 0
                for word in words:
                    if '/' in word:
                        if len(word.split('/')[0]) > 1 and len(word.split('/')[-1]) > 1:
                            sector = word
                            flag = 1
                            sectors.append(sector)
                            break
                x = re.search(".*sub.*total.*", text[j].lower())
                if x or 'inr' in text[j].lower() or 'terms & conditions' in text[j].lower() or 'processing charges' in text[j].lower():
                    if sectors:
                        # print('sectors---> ', sectors)
                        # if not sectors[0]:
                        #     print('sectors----> ', sectors)
                        return sectors[:l]
    if not flag:
        # if not sectors[0]:
            # print('sectors----> ', sectors)
        sectors.extend(['None']*l)
        # print('sectors none---> ', sectors)
        return sectors 

def divide_sectors(sectors):
    src = []
    dest = []
    for sector in sectors:
        if '/' in sector:
            l = sector.split("/")
        else:
            l = sector.split()
        s = []
        d = []
        for i in range(len(l)-1):
            s.append(l[i].strip())
            d.append(l[i+1].strip())
        src.append(s)
        dest.append(d)
    
    # print('src---> ', src)
    # print('dest---> ', dest)
    return src, dest

def get_city_names(sectors):
    cities = []
    countries = []
    
    for sector in sectors:
        try:
            # print(code)
            value = data[data['iata'] == sector.strip().upper()].iloc[0]
            # print('value---> ', value)
            cities.append(value['city'])
            countries.append(value['country'])
        except Exception as e:
            cities.append('None') 
            countries.append('None')
    
    return cities, countries

def get_invoice(text):
    invoice = []
    inv = ''
    for i in range(len(text)):
        if 'tax & chrg' in text[i].lower() or 'sectors' in text[i].lower() or 'basic fare' in text[i].lower():
            words = text[i+1].split()
            for j in range(len(words)):
                if ('ds' in words[j].lower() or 'ps' in words[j].lower()) and not words[j].isalpha():
                    inv = 'DS_'+words[j+1]
                    break
                elif ('is' in words[j].lower() or '18' in words[j].lower() or '1s' in words[j].lower()) and not words[j].isalpha() and len(words[j]) < 7:
                    inv = 'IS_'+words[j+1]
                    break
        if inv:
            invoice.append(inv)
            return invoice

def get_invoice_date(text):
    for i in range(len(text)):
        if 'crn.dt.' in text[i].lower() or re.search('inv.*.dt.', text[i].lower()):
            words = text[i].split()
            return words[-1]

traveler_names_sheet = []
sectors_sheet = []
start_dates_sheet = []
end_dates_sheet = []
source_cities_sheet = []
dest_cities_sheet = []
source_countries_sheet = []
dest_countries_sheet = []
filename_sheet = []
supplier_sheet = []
invoice_date_sheet = []
invoice_sheet = []

data = pd.read_excel('World_DB.xlsx')

count = 0
for file in glob.glob('C:/matsya/ensemble_files_apr_24/*.txt'):
    # print(file)
    traveler_names_final = []
    start_dates_final = []
    end_dates_final = []
    source_cities_final = []
    dest_cities_final = []
    source_countries_final = []
    dest_countries_final = []
    sectors_final = []
    invoice_date_final = []
    invoice_final = []

    with open(file) as fp:

        filename = os.path.split(file)[-1]
        words = filename.split('_')
        for i in range(len(words)):
            if 'page' in words[i]:
                ind = i
                break
        filename = '_'.join(words[:ind])
        # print(filename)

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            filename = os.path.split(file)[-1].split('.txt')[0]

            traveler_names = get_traveler_names(text)
            if traveler_names and not traveler_names_final:
                traveler_names_final = traveler_names

            start_dates = get_start_dates(text)
            if start_dates and not start_dates_final:
                start_dates_final = start_dates

            end_dates = get_start_dates(text)
            if end_dates and not end_dates_final:
                end_dates_final = end_dates

            sectors = get_sectors(text)
            if sectors and not sectors_final:
                sectors_final = sectors
            
            # div_sectors = divide_sectors(sectors)
            # print('sectors---> ', sectors)
            source_cities = divide_sectors(sectors)[0]
            if source_cities and not source_cities_final:
                source_cities_final = source_cities

            dest_cities = divide_sectors(sectors)[1]
            if dest_cities and not dest_cities_final:
                dest_cities_final = dest_cities

            inv_date = get_invoice_date(text)
            # print('inv_date---> ', inv_date)
            if inv_date and not invoice_date_final:
                invoice_date_final = inv_date
                # print(invoice_date_final)

            if 'ir' in filename.lower() or 'dr' in filename.lower():
                # count += 1
                # print(filename)
                if 'ir_217' in filename.lower():
                    invoice_final = ['IS_1105']
                if 'ir_406' in filename.lower():
                    invoice_final = ['IS_2039']
                if 'ir_236' in filename.lower():
                    invoice_final = ['IS_1096']
                if 'ir_237' in filename.lower():
                    invoice_final = ['IS_1316']
                if 'ir_240' in filename.lower():
                    invoice_final = ['IS_1325']
                if 'ir_275' in filename.lower():
                    invoice_final = ['IS_1504']
                if 'ir_340' in filename.lower():
                    invoice_final = ['IS_1753']
                if 'ir_385' in filename.lower():
                    invoice_final = ['IS_1889']
                if 'ir_387' in filename.lower():
                    invoice_final = ['IS_1923']
                if 'ir_421' in filename.lower():
                    invoice_final = ['IS_2091']
                invoice = get_invoice(text)
                if invoice and not invoice_final:
                    invoice_final = invoice
            else:
                invoice = ['None']
                if invoice and not invoice_final:
                    invoice_final = invoice
            
            # source_cities = get_source_cities(text)[0]
            # if source_cities and not source_cities_final:
            #     source_cities_final = source_cities

            # dest_cities = get_destination_cities(text)[0]
            # if dest_cities and not dest_cities_final:
            #     dest_cities_final = dest_cities

            # source_countries = get_source_cities(text)[1]
            # if source_countries and not source_countries_final:
            #     source_countries_final = source_countries

            # dest_countries = get_destination_cities(text)[1]
            # if dest_countries and not dest_countries_final:
            #     dest_countries_final = dest_countries

        # if len(traveler_names_final) < len(start_dates_final):
            # traveler_names_final.extend([None]*(len(start_dates_final)-len(traveler_names_final)))
        # traveler_names_sheet.extend(traveler_names_final)
        # sectors_sheet.extend(sectors_final)
        # source_cities_sheet.append(source_cities_final)
        # dest_cities_sheet.append(dest_cities_final)


        # if len(start_dates_final) < len(traveler_names_final):
        #     start_dates_final.extend([None]*(len(traveler_names_final)-len(start_dates_final)))
        # start_dates_sheet.extend(start_dates_final)
        # if len(end_dates_final) < len(traveler_names_final):
        #     end_dates_final.extend([None]*(len(traveler_names_final)-len(end_dates_final)))
        # end_dates_sheet.extend(end_dates_final)
        # if len(source_cities_final) < len(traveler_names_final):
        #     source_cities_final.extend([None]*(len(traveler_names_final)-len(source_cities_final)))
        # source_cities_sheet.extend(source_cities_final)
        # if len(dest_cities_final) < len(traveler_names_final):
        #     dest_cities_final.extend([None]*(len(traveler_names_final)-len(dest_cities_final)))
        # dest_cities_sheet.extend(dest_cities_final)
        # if len(source_countries_final) < len(traveler_names_final):
        #     source_countries_final.extend([None]*(len(traveler_names_final)-len(source_countries_final)))
        # source_countries_sheet.extend(source_countries_final)
        # if len(dest_countries_final) < len(traveler_names_final):
        #     dest_countries_final.extend([None]*(len(traveler_names_final)-len(dest_countries_final)))
        # dest_countries_sheet.extend(dest_countries_final)

        # print(file)
        # print(traveler_names_final)
        # print(source_cities_final)
        # print(dest_cities_final)
        # print(start_dates_final)
        # print(end_dates_final)
        # print()

        # if not source_cities_final[0]:
        #     for i in range(len(source_cities_final)):
        #         source_cities_final[i] = ['None']
        
        # if not dest_cities_final[0]:
        #     for i in range(len(dest_cities_final)):
        #         dest_cities_final[i] = ['None']

        # if not invoice_final:
        #     print('Filename---> ', filename)
        #     print('Traveler Name---> ', traveler_names_final)
        #     print('Start Date---> ', start_dates_final)
        #     print('End Date---> ', end_dates_final)
        #     print('Source City---> ', source_cities_final)
        #     print('Destination City---> ', dest_cities_final)
        #     print('Invoice Final---> ', invoice_final)
        #     print('--------------------------------------------------')

        ## PRINT ALL RECORDS
        
        # if not invoice_date_final:
            # count += 1
        print('Filename---> ', filename)
        print('Traveler Name---> ', traveler_names_final)
        print('Start Date---> ', start_dates_final)
        print('End Date---> ', end_dates_final)
        print('Source City---> ', source_cities_final)
        print('Destination City---> ', dest_cities_final)
        print('Invoice Final---> ', invoice_final)
        print('Invoice Date---> ', invoice_date_final)
        print('--------------------------------------------------')
# print(count)

        # print(traveler_names_final)
        tnames_total = []
        # l=0
        # print(filename)
        for i in range(len(traveler_names_final)):

            if len(source_cities_final) < len(traveler_names_final) or len(start_dates_final) < len(traveler_names_final):
                source_cities_final.append(['None'])
                dest_cities_final.append(['None'])
                start_dates_final.append('None')
                end_dates_final.append('None')
            
            if source_cities_final:
                # print(i)
                # print('traveler names final---> ', traveler_names_final[i])
                # print('source cities final---> ', source_cities_final[i])
                # print('dest cities final---> ', dest_cities_final[i])
                # print('start_dates_final---> ', start_dates_final[i])
                # print('end dates final---> ', end_dates_final[i])
                l = len(source_cities_final[i])
            
            fn = filename.split('_page')[0]
            filename_sheet.extend([fn]*l)
            if traveler_names_final:
                traveler_names_sheet.extend([traveler_names_final[i]]*l)
                tnames_total = [traveler_names_final[i]]*l
            if invoice_final:
                invoice_sheet.extend(invoice_final*len(tnames_total))
            if source_cities_final:
                source_cities_sheet.extend(source_cities_final[i])
            if dest_cities_final:
                dest_cities_sheet.extend(dest_cities_final[i])
            if start_dates_final:
                start_dates_sheet.extend([start_dates_final[i]]*l)
            if end_dates_final:
                end_dates_sheet.extend([end_dates_final[i]]*l)
            if invoice_date_final:
                # print('l---> ', l)
                # print(invoice_date_final)
                invoice_date_sheet.extend([invoice_date_final]*l)
        # print('------------')

        # print('tnames total---> ', tnames_total)
        # count = len(tnames_total)
        # print('count---> ', count)
            
        # invoice_final_each = invoice_final*count
        # if 'dr' in filename.lower() or 'ir' in filename.lower():
        #     print(file)
        #     print(invoice_final_each)
        # invoice_sheet.extend(invoice_final_each)
            
        
        # for i in range(len(start_dates_final)):
        #     l = len(source_cities_final[i])
        #     start_dates_sheet.extend([start_dates_final[i]]*l)
        #     end_dates_sheet.extend([end_dates_final[i]]*l)
    
    # if not source_cities_final[0]:
    #     print(file)
    #     print(traveler_names_final)
    #     print(source_cities_final)
    #     print(dest_cities_final)
    #     print(start_dates_final)
    #     print(end_dates_final)
    #     print()
    
# print(source_cities_sheet)

# filename_sheet.extend([filename]*len(traveler_names_sheet))
supplier_sheet.extend(['MATSYA TRAVEL SERVICES PVT. LTD.']*len(traveler_names_sheet))
if source_cities_sheet:
    source_cities_sheet_final = get_city_names(source_cities_sheet)[0]
if dest_cities_sheet:
    dest_cities_sheet_final = get_city_names(dest_cities_sheet)[0]
if source_cities_sheet:
    source_countries_sheet = get_city_names(source_cities_sheet)[1]
if dest_cities_sheet:
    dest_countries_sheet = get_city_names(dest_cities_sheet)[1]

# print(source_cities_sheet)
# print(source_cities_sheet_final)

print(len(filename_sheet))
print(len(traveler_names_sheet))
print(len(start_dates_sheet))
print(len(end_dates_sheet))
print(len(source_cities_sheet_final))
print(len(dest_cities_sheet_final))
print(len(source_countries_sheet))
print(len(dest_countries_sheet))
print(len(supplier_sheet))
print(len(invoice_date_sheet))
print(len(invoice_sheet))
# print(invoice_date_sheet)

Filename--->  DR_12_page0
Traveler Name--->  ['KARISHMA NEELESH']
Start Date--->  ['2024-04-22']
End Date--->  ['2024-04-22']
Source City--->  [['JAI']]
Destination City--->  [['BLR']]
Invoice Final--->  ['DS_16']
Invoice Date--->  22-04-2024
--------------------------------------------------
Filename--->  DR_1_page0
Traveler Name--->  ['SUBRATO MUKHERJEE', 'SUBRATO MUKHERJEE']
Start Date--->  ['2024-04-07', '2024-04-07']
End Date--->  ['2024-04-07', '2024-04-07']
Source City--->  [['PNQ'], ['DEL']]
Destination City--->  [['DEL'], ['PNQ']]
Invoice Final--->  ['DS_1560']
Invoice Date--->  07-04-2024
--------------------------------------------------
Filename--->  DR_3_page0
Traveler Name--->  ['ABHIJIT DILIP SANE']
Start Date--->  ['2024-04-07']
End Date--->  ['2024-04-07']
Source City--->  [['DEL']]
Destination City--->  [['PNQ']]
Invoice Final--->  ['DS_1537']
Invoice Date--->  07-04-2024
--------------------------------------------------
Filename--->  DR_4_page0
Traveler Name--->  ['

In [6]:
############ create dataframe for all records #################

def df_for_all_records():
    all_invoices = pd.DataFrame({
        'Filename': filename_sheet,
        'Invoice Date': invoice_date_sheet,
        'Traveler Name': traveler_names_sheet,
        'Start Date': start_dates_sheet,
        'End Date': end_dates_sheet,
        'Source City': source_cities_sheet_final,
        'Source Country': source_countries_sheet,
        'Destination City': dest_cities_sheet_final,
        'Destination Country': dest_countries_sheet,
        'Supplier Name': supplier_sheet,
        'Associated Invoice': invoice_sheet
    })

    return all_invoices

In [7]:
############## create dataframe for credit notes ###############

all_invoices = df_for_all_records()
all_invoices.to_excel('apr_24_invoices.xlsx')

def df_for_cred(all_invoices):
    cred_notes = pd.DataFrame()
    cred_notes = all_invoices.loc[(all_invoices['Associated Invoice'] != 'None')]
    return cred_notes

cred_notes = df_for_cred(all_invoices)
cred_notes.to_excel('apr_24_cred_notes.xlsx')

In [8]:
cred_filename = list(cred_notes['Filename'])
# print(len(cred_filename))
ass_inv = list(cred_notes['Associated Invoice'])
# print(len(ass_inv))

removed_invoices = all_invoices[~all_invoices['Filename'].isin(cred_filename)]
removed_invoices = removed_invoices[~removed_invoices['Filename'].isin(ass_inv)]
removed_invoices
 
removed_invoices.to_excel('apr_24_final_invoices.xlsx')

In [8]:
## ques

a = [2,1]

prod = a[0]

for i in range(len(a)-1):
    k = a[i+1]
    sum = 0
    for j in range(k):
        sum += prod
    prod = sum

print(prod)

2


In [4]:
1*2*3*4*5

120

In [4]:
a = [7,2,3,1,9,4]
mul = [7,2,3,1,9,4]
digits = []

while True:
    mul = [i*2 for i in mul]
[]

KeyboardInterrupt: 

In [12]:
a = [7,2,1,8]
new = a

while True:
    for i in range(len(a)):
        for j in range(len(a))
        if new[i] != 0 and '-' not in str(new[i]):
            sub = new[i]
        new[i] -= sub
        print(new)
    break

print(new)

[0, 2, 1, 8]
[0, 0, 1, 8]
[0, 0, 0, 8]
[0, 0, 0, 0]
[0, 0, 0, 0]


In [6]:
# selection sort

a = [6,1,3,4,9,2,10,13,5]
# min_ind = 0
for i in range(len(a)):
    min_ind = i
    for j in range(i+1, len(a)):
        if a[j] < a[min_ind]:
            min_ind = j
    a[i], a[min_ind] = a[min_ind], a[i]
print(a)    


[1, 2, 3, 4, 5, 6, 9, 10, 13]


In [15]:
n=10
for i in range(n):
    print(' '*(n-1-i), end='')
    print('*'*((2*i)+1))

         *
        ***
       *****
      *******
     *********
    ***********
   *************
  ***************
 *****************
*******************
